In [1]:
import ast
import atexit
import io
import itertools
import json
import os
import re
import sys
import uuid
from datetime import datetime

from IPython.display import HTML, display

if "spark" not in vars():
    import findspark

    findspark.init()
    import pyspark.sql.functions as F
    import pyspark.sql.types as T
    from pyspark.sql import DataFrame, SparkSession
    from pyspark.sql.window import Window
    from pyspark.storagelevel import StorageLevel

    spark = (
        SparkSession.builder.master("local[8,2]")
        .config("spark.driver.memory", "4g")
        .enableHiveSupport()
        .getOrCreate()
    )
    atexit.register(lambda: spark.stop())

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import DataFrame, Row, SparkSession
from pyspark.sql.window import Window


def cleanse_val(val):
    return re.sub(r"([^\s\w\d])+", "", val.lower()).strip() if val else ""


def reg(spark_df, name=None):
    uniqsig = "df_{0}".format(cleanse_val(str(uuid.uuid4()))) if not name else name
    spark_df.createOrReplaceTempView(uniqsig)
    return uniqsig


def show(df, rows=5):
    display(df.limit(rows).toPandas())


# Override table show/registration functions
DataFrame.reg = reg
DataFrame.dshow = show

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/07 23:47:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/07 23:47:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/07 23:47:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
import glob
from pathlib import Path

for file in glob.glob("data/*.csv"):
    df = (
        spark.read.option("header", True)
        .option("inferSchema", True)
        .csv(f"file://{os.getcwd()}/{file}")
    )
    df.dshow(5)
    print(f"default.{Path(Path(file).name).stem}")
    df.write.saveAsTable(f"{Path(file).stem.strip().title()}", mode="ignore")

24/03/07 23:47:52 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


Symbol                 Name       Sector
0    MMM                   3M  Industrials
1    AOS          A. O. Smith  Industrials
2    ABT  Abbott Laboratories  Health Care
3   ABBV               AbbVie  Health Care
4   ABMD              Abiomed  Health Care

default.s_and_p_mapping


24/03/07 23:47:57 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


date   open   high    low  close    volume Name
0  2013-02-08  15.07  15.12  14.63  14.75   8407500  AAL
1  2013-02-11  14.89  15.01  14.26  14.46   8882000  AAL
2  2013-02-12  14.45  14.51  14.10  14.27   8126000  AAL
3  2013-02-13  14.30  14.94  14.25  14.66  10259500  AAL
4  2013-02-14  14.94  14.96  13.16  13.99  31879900  AAL

default.s_and_p_5_years


id ident           type                  name  latitude_deg  \
0    6523   00A       heliport     Total RF Heliport     40.070985   
1  323361  00AA  small_airport  Aero B Ranch Airport     38.704022   
2    6524  00AK  small_airport          Lowell Field     59.947733   
3    6525  00AL  small_airport          Epps Airpark     34.864799   
4  506791  00AN  small_airport  Katmai Lodge Airport     59.093287   

   longitude_deg  elevation_ft continent iso_country iso_region  municipality  \
0     -74.933689            11        NA          US      US-PA      Bensalem   
1    -101.473911          3435        NA          US      US-KS         Leoti   
2    -151.692524           450        NA          US      US-AK  Anchor Point   
3     -86.770302           820        NA          US      US-AL       Harvest   
4    -156.456699            80        NA          US      US-AK   King Salmon   

  scheduled_service gps_code iata_code local_code  \
0                no     K00A      None        00A   
1                no     00AA      None       00AA   
2                no     00AK      None       00AK   
3                no     00AL      None       00AL   
4                no     00AN      None       00AN   

                                           home_link wikipedia_link keywords  
0  https://www.penndot.pa.gov/TravelInPA/airports...           None     None  
1                                               None           None     None  
2                                               None           None     None  
3                                               None           None     None  
4                                               None           None     None

default.airports


In [3]:
import torch
torch.cuda.is_available()

True